In [2]:
import numpy as np 
import pandas as pd 
from pathlib import Path

from sklearn.svm import LinearSVC

# Feature creation

Both algorithms use standard bag-of-words features with stop word removal and optional tf–idf weighting.

## BOW and Tf-idf

described at: https://medium.com/betacom/bow-tf-idf-in-python-for-unsupervised-learning-task-88f3b63ccd6d

## Stop word removal



# Algorithms



Reduction to Binary classifyers:

implementation of liblinear in sklearn is used which can be found in: https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC
description at: https://scikit-learn.org/stable/modules/svm.html (chapter 1.4.1.1.)

use with:
LinearSVC(random_state=0, tol=1e-5)

# Load Data

todo: cite and description of data

In [3]:
DATA_FILES = Path('data')
TEST_DATA = DATA_FILES / 'test.txt'
TRAIN_DATA = DATA_FILES / 'train.txt'
print(TEST_DATA)
print(TRAIN_DATA)

data/test.txt
data/train.txt


In [7]:
def load_data(file_path):
    X = []
    y = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
    for line in lines:
        sentence = ' '.join(line.split()[0:-1])
        X.append(sentence)
        labels = line.split()[-1].split('_')
        y.append(labels)
    return X, y
        
X_train, y_train = load_data(TRAIN_DATA)
X_test, y_test = load_data(TEST_DATA)